# VCF to Gene Expression Prediction

This notebook demonstrates how to use the VariantFormer VCFProcessor to predict gene expression from VCF (Variant Call Format) files. The VCFProcessor leverages a transformer model to predict gene expression levels based on genetic variants and tissue types.

## Overview

The VariantFormer pipeline can:
- Process VCF files containing genetic variants
- Predict gene expression levels for specific genes and tissues
- Generate embeddings for specific genes
- Handle multiple tissues and genes in a single analysis

## Prerequisites

- GPU-enabled environment (CUDA required)
- Access to reference genome and model checkpoints (run `python download_artifacts.py` before running the notebook)
- VCF files with genetic variants

## Key Outputs:

- **Gene Expression Predictions**: Quantitative predictions of expression levels
- **Embeddings**: Embedding for each gene-tissue pair


In [1]:
import sys
import os
from pathlib import Path
import ipynbname
import pandas as pd
from processors.vcfprocessor import VCFProcessor
import warnings
 
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"🚀 GPU available: {torch.cuda.get_device_name(0)}")
    print(
        f"💾 GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB"
    )
else:
    print("⚠️  No GPU available - this notebook requires CUDA")

# Set repository path
REPO_PATH = ipynbname.path().parent.parent

2025-11-04 15:09:07 - matplotlib - DEBUG - matplotlib data path: /work/.venv/lib/python3.12/site-packages/matplotlib/mpl-data
2025-11-04 15:09:07 - matplotlib - DEBUG - CONFIGDIR=/flow/.config/matplotlib
2025-11-04 15:09:07 - matplotlib - DEBUG - interactive is False
2025-11-04 15:09:07 - matplotlib - DEBUG - platform is linux
2025-11-04 15:09:08 - matplotlib - DEBUG - CACHEDIR=/flow/.cache/matplotlib
2025-11-04 15:09:08 - matplotlib.font_manager - DEBUG - Using fontManager instance from /flow/.cache/matplotlib/fontlist-v390.json


🚀 GPU available: NVIDIA H100 80GB HBM3
💾 GPU memory: 84.9 GB


## 1. Initialize VCFProcessor

The VCFProcessor is the main class that handles:
- Loading model configurations
- Managing tissue and gene vocabularies  
- Creating data loaders for VCF files
- Loading pre-trained models (v4_pcg, v4_ag)
- Running predictions

Let's initialize it with the default VariantFormer protein-coding gene (PCG) model.


In [2]:
# Initialize the VCFProcessor with the default v4_PCG model
model_class = "v4_ag" # You can also use "v4_ag" for protein-coding genes
#model_class = "v4_pcg"  # Uncomment to use the protein-coding gene model
vcf_processor = VCFProcessor(model_class=model_class)

print(f"📊 Model class: {model_class}")
print(f"⚙️  Configuration loaded from: {vcf_processor.config_location}")

📊 Model class: v4_ag
⚙️  Configuration loaded from: /work/configs


## 2. Explore Available Tissues and Genes

Before making predictions, let's explore what tissues and genes are available in the system. This will help us understand the scope of the model and choose appropriate targets for our analysis.


In [3]:
# Get available tissues
tissues = vcf_processor.get_tissues()
print(f"🧪 Available tissues ({len(tissues)} total):")
print("=" * 50)
print("First 10 tissues in the dataset:")
for i, tissue in enumerate(tissues, 1):
    print(f"{i:2d}. {tissue}")
    if i == 10:
        break

print("\n" + "=" * 70)

# Get available genes
genes_df = vcf_processor.get_genes()
print(f"🧬 Available genes ({len(genes_df)} total):")
print("=" * 50)
print("First 10 genes in the dataset:")
print(genes_df[["gene_id", "gene_name"]].head(10).to_string(index=False))

print("\n📊 Gene statistics:")
print(f"   • Total genes: {len(genes_df):,}")

🧪 Available tissues (62 total):
First 10 tissues in the dataset:
 1. A549
 2. GM23248
 3. HepG2
 4. K562
 5. NCI-H460
 6. Panc1
 7. adipose - subcutaneous
 8. adipose - visceral (omentum)
 9. adrenal gland
10. artery - aorta

🧬 Available genes (51061 total):
First 10 genes in the dataset:
           gene_id gene_name
ENSG00000000419.12      DPM1
ENSG00000000457.13     SCYL3
ENSG00000000460.16  C1orf112
ENSG00000000938.12       FGR
ENSG00000000971.15       CFH
ENSG00000001036.13     FUCA2
ENSG00000001084.10      GCLC
ENSG00000001167.14      NFYA
ENSG00000001460.17     STPG1
ENSG00000001461.16    NIPAL3

📊 Gene statistics:
   • Total genes: 51,061


## 3. Prepare Query Data

Now we'll prepare our query data, which specifies:
- **gene_id**: Which genes we want to predict expression for
- **tissues**: Which tissues/cell types we're interested in

For this demo, we'll use the same example as in the test function, but let's also prepare a more diverse example.


In [4]:
# Example 1: Simple query
'''
General format for queries:
simple_query = {
    "gene_id": [gene ID],
    "tissues": [Comma separated tissue names],
}
'''
simple_query = {
    "gene_id": ["ENSG00000001461.16", "ENSG00000000419.12"],
    "tissues": ["whole blood,thyroid,artery - aorta", "brain - amygdala"],
} 
query_df = pd.DataFrame(simple_query)
print("🔍 Simple Query DataFrame:")
print(query_df.to_string(index=False))

🔍 Simple Query DataFrame:
           gene_id                            tissues
ENSG00000001461.16 whole blood,thyroid,artery - aorta
ENSG00000000419.12                   brain - amygdala


## 4. Specify VCF File and Create Dataset

Now we need to specify the path to our VCF file containing genetic variants. The VCFProcessor will:
- Load the VCF file and extract relevant variants
- Map variants to regulatory regions (CREs) and genes
- Create sequence data for model input
- Prepare batches for efficient processing

**Note**: Update the `vcf_path` below to point to your actual VCF file.


In [5]:
# Specify VCF file path (update this path to your actual VCF file)
vcf_path = os.path.join(REPO_PATH, "_artifacts/HG00096.vcf.gz")
# Create data loader from VCF and query
vcf_dataset, dataloader = vcf_processor.create_data(vcf_path, query_df)

Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500.json
Filtered query df to 2 genes reducing from 2


In [6]:
import time

print("🔄 Loading pre-trained model...")
start_time = time.time()


model, checkpoint_path, trainer = vcf_processor.load_model()

load_time = time.time() - start_time
print(f"📂 Checkpoint path: {checkpoint_path}")
print(f"⚡ Precision: {trainer.precision}")

# Print model information
total_params = sum(p.numel() for p in model.parameters())

print("\n📊 Model Statistics:")
print(f"   • Total parameters: {total_params:,}")


2025-11-04 15:09:21 - processors.model_manager - INFO - Loading Seq2Reg model...


🔄 Loading pre-trained model...


2025-11-04 15:09:22 - processors.model_manager - INFO - Loading Seq2Reg gene model...
2025-11-04 15:09:22 - processors.model_manager - INFO - Creating Seq2Gene model...
2025-11-04 15:09:28 - processors.model_manager - INFO - Model class: <class 'seq2gene.model_combined_modulator.Seq2GenePredictorCombinedModulator'>
2025-11-04 15:09:28 - processors.model_manager - INFO - Model architecture:
2025-11-04 15:09:28 - processors.model_manager - INFO - Model: Seq2GenePredictorCombinedModulator
2025-11-04 15:09:28 - processors.model_manager - INFO -   start_tkn: 96,768 params
2025-11-04 15:09:28 - processors.model_manager - INFO -   cre_tokenizer: 31,826,153 params
2025-11-04 15:09:28 - processors.model_manager - INFO -   gene_tokenizer: 31,826,153 params
2025-11-04 15:09:28 - processors.model_manager - INFO -   gene_map: 787,968 params
2025-11-04 15:09:28 - processors.model_manager - INFO -   cre_map: 787,968 params
2025-11-04 15:09:28 - processors.model_manager - INFO -   combined_modulator: 

📂 Checkpoint path: /work/_artifacts/v4_ag_epoch9_checkpoint.pth
⚡ Precision: bf16-mixed

📊 Model Statistics:
   • Total parameters: 1,227,349,459


## 6. Run Predictions

Now we're ready to run the actual predictions! The model will:
- Process the genetic variants 
- Generate embeddings that capture gene representation
- Predict gene expression levels for each gene-tissue combination

This step may take some time depending on the size of your VCF file and the complexity of your queries. Prune the VCF files to remove all ref variant which will improve speed.


In [7]:
print("🔄 Running predictions...")

start_time = time.time()

# Run predictions
predictions_df = vcf_processor.predict(
    model, checkpoint_path, trainer, dataloader, vcf_dataset
)

prediction_time = time.time() - start_time
print(f"✅ Predictions completed in {prediction_time:.1f} seconds!")
print(f"📊 Results shape: {predictions_df.shape}")



INFO: Restoring states from the checkpoint path at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:09:54 - lightning.pytorch.utilities.rank_zero - INFO - Restoring states from the checkpoint path at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:09:54 - fsspec.local - DEBUG - open file: /work/_artifacts/v4_ag_epoch9_checkpoint.pth


🔄 Running predictions...


/work/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:283: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': None, 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}"].
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025-11-04 15:10:01 - lightning.pytorch.accelerators.cuda - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:10:01 - lightning.pytorch.utilities.rank_zero - INFO - Loaded model weights from the checkpoint at /work/_artifacts/v4_ag_epoch9_checkpoint.pth


Predicting: |          | 0/? [00:00<?, ?it/s]

2025-11-04 15:10:02 - utils.assets - INFO - Using cached file: /work/_artifacts/model/common/cres_all_genes_manifest.parquet
2025-11-04 15:10:02 - utils.assets - INFO - Loading parquet file: /work/_artifacts/model/common/cres_all_genes_manifest.parquet
2025-11-04 15:10:02 - utils.assets - INFO - Validated schema - found columns: {'gene_id', 'file_path'}
2025-11-04 15:10:02 - filelock - DEBUG - Attempting to acquire lock 139332232152368 on /work/_artifacts/model/common/cres_all_genes/ENSG00000001461.16/gene_vocab.csv.lock
2025-11-04 15:10:02 - filelock - DEBUG - Lock 139332232152368 acquired on /work/_artifacts/model/common/cres_all_genes/ENSG00000001461.16/gene_vocab.csv.lock
2025-11-04 15:10:02 - asyncio - DEBUG - Using selector: EpollSelector
2025-11-04 15:10:02 - s3fs - DEBUG - Setting up s3fs instance
2025-11-04 15:10:02 - botocore.hooks - DEBUG - Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
2025-11-04 15:10:02 - botocore.hooks - D

✅ Predictions completed in 29.7 seconds!
📊 Results shape: (2, 5)


## 7. Analyze Results

In [8]:
print("📊 PREDICTION RESULTS ANALYSIS")
print("=" * 50)

# Display basic information about results
print("🔍 Results Overview:")
print(f"   • Number of predictions: {len(predictions_df)}")
print(f"   • Columns: {list(predictions_df.columns)}")
print("\n📋 Sample Results:")
print("=" * 30)
print("Predictions:")
predictions_df

📊 PREDICTION RESULTS ANALYSIS
🔍 Results Overview:
   • Number of predictions: 2
   • Columns: ['gene_id', 'tissues', 'tissue_names', 'predicted_expression', 'embeddings']

📋 Sample Results:
Predictions:


,gene_id,tissues,tissue_names,predicted_expression,embeddings
0,ENSG00000001461.16,"[62, 59, 10]","[whole blood, thyroid, artery - aorta]","[[1.2623504], [2.944679], [1.612904]]","[[-6.375, -4.78125, 1.5546875, 2.71875, 0.4042..."
1,ENSG00000000419.12,[15],[brain - amygdala],[[2.8708205]],"[[9.625, 4.65625, -3.34375, 1.90625, 19.625, -..."


**The output schema**
- Original query information: (gene_id, tissues)
- `predicted_expression`: Model's prediction of gene expression levels
- `embeddings`: High-dimensional representations of the gene conditioned on tissue and neighboring regulatory regions

## Run the analysis on ref genome hg38 without any mutations
This analysis allows to compare gene expression change due to the presence of the mutations.

In [9]:
# Np VCF file provided, so using empty string
vcf_dataset, dataloader = vcf_processor.create_data("", query_df) # Replace actual VCF path with "" to indicate no VCF
predictions_df_ref = vcf_processor.predict(
    model, checkpoint_path, trainer, dataloader, vcf_dataset
)

INFO: Restoring states from the checkpoint path at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:10:40 - lightning.pytorch.utilities.rank_zero - INFO - Restoring states from the checkpoint path at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:10:40 - fsspec.local - DEBUG - open file: /work/_artifacts/v4_ag_epoch9_checkpoint.pth


Loaded BPE vocabulary from /work/vocabs/bpe_vocabulary_500.json
Filtered query df to 2 genes reducing from 2


/work/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:283: Be aware that when using `ckpt_path`, callbacks used to create the checkpoint need to be provided during `Trainer` instantiation. Please add the following callbacks: ["ModelCheckpoint{'monitor': None, 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}"].
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2025-11-04 15:10:46 - lightning.pytorch.accelerators.cuda - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /work/_artifacts/v4_ag_epoch9_checkpoint.pth
2025-11-04 15:10:46 - lightning.pytorch.utilities.rank_zero - INFO - Loaded model weights from the checkpoint at /work/_artifacts/v4_ag_epoch9_checkpoint.pth


Predicting: |          | 0/? [00:00<?, ?it/s]

2025-11-04 15:10:47 - utils.assets - INFO - Using cached file: /work/_artifacts/model/common/cres_all_genes_manifest.parquet
2025-11-04 15:10:48 - utils.assets - INFO - Loading parquet file: /work/_artifacts/model/common/cres_all_genes_manifest.parquet
2025-11-04 15:10:48 - utils.assets - INFO - Validated schema - found columns: {'gene_id', 'file_path'}
2025-11-04 15:10:48 - utils.assets - INFO - Using cached file: /work/_artifacts/model/common/cres_all_genes/ENSG00000001461.16/gene_vocab.csv
2025-11-04 15:10:49 - utils.assets - INFO - Using cached file: /work/_artifacts/model/common/cres_all_genes/ENSG00000000419.12/gene_vocab.csv


In [10]:
print("✅ Predictions with made from reference genome (no VCF provided):")  
predictions_df_ref.head()

✅ Predictions with made from reference genome (no VCF provided):


,gene_id,tissues,tissue_names,predicted_expression,embeddings
0,ENSG00000001461.16,"[62, 59, 10]","[whole blood, thyroid, artery - aorta]","[[1.1905905], [2.7972884], [1.6317137]]","[[-5.71875, -4.65625, 1.0859375, 1.9765625, -1..."
1,ENSG00000000419.12,[15],[brain - amygdala],[[2.8560872]],"[[9.625, 3.453125, -3.703125, 2.328125, 18.875..."


### Compare the log2fc from REF genome

In [11]:
import numpy as np

# Extract the predicted expression values from nested lists
ref_expr = predictions_df_ref['predicted_expression'].values[0]
vcf_expr = predictions_df['predicted_expression'].values[0]

# Calculate log2 fold change
print(f'Tissues: {predictions_df["tissue_names"].values[0]}')
print(f'Log2 fold change: {np.log2(vcf_expr / ref_expr)}')

Tissues: ['whole blood', 'thyroid', 'artery - aorta']
Log2 fold change: [[ 0.0844352 ]
 [ 0.07408135]
 [-0.0167275 ]]


In [12]:
predictions_df

,gene_id,tissues,tissue_names,predicted_expression,embeddings
0,ENSG00000001461.16,"[62, 59, 10]","[whole blood, thyroid, artery - aorta]","[[1.2623504], [2.944679], [1.612904]]","[[-6.375, -4.78125, 1.5546875, 2.71875, 0.4042..."
1,ENSG00000000419.12,[15],[brain - amygdala],[[2.8708205]],"[[9.625, 4.65625, -3.34375, 1.90625, 19.625, -..."
